# Fidelity Training Script

This script trains the convolution neural network to recognize $\mathcal{F}$ values for every number of bases employed in compressive tomography, and later predicts the values with test examples. The complete simulated data consist of those from both statistically noiseless and noisy cases of ACT and random Haar schemes (4 batches). Each batch consists of $m=5000$ training datasets generated from a separate MATLAB script that simulates the ACT and random Haar schemes on 4-qubit systems ($d=2^4=16$).

### Loads packages.

In [1]:
import scipy.io as spi
import os

import tensorflow as tf
import numpy as np
import pandas as pd
import seaborn as sns
import itertools
import json
import h5py

from keras.layers import Concatenate, Dense, Dropout, Conv2D, MaxPooling2D, Flatten, merge, Input, ZeroPadding2D, Activation, Add, AveragePooling2D
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.models import Sequential, load_model, Model
from keras import optimizers, regularizers
from keras import backend as K
from keras.layers.normalization import BatchNormalization

from sklearn import preprocessing
from sklearn.model_selection import train_test_split

/home/ys_teo/ysteo-tensorflow_old/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/ys_teo/ysteo-tensorflow_old/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/ys_teo/ysteo-tensorflow_old/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/ys_teo/ysteo-tensorflow_old/lib/python3.5/s

### Defines work parameters

In [2]:
d=16 # Hilbert-space dimension
m=8000 # Number of training datasets
num_classes=1 # Set to 1 by default since we are fitting a single continuous output value

# Defines the working directory containing all generated MATLAB MAT training data files
work_dir='../training/'+str(m)+'_perf_noisy_ex/'

# Defines ICCNet model storage directory
stor_dir=work_dir+'FidNet_trained_files/'

### Training commences

In [3]:
# Loops over different number of bases "Kbas" for training FidNet

for l in np.linspace(1,10,10): # Defines the number of bases "Kbas" for training the ConvNet.

    Kbas=int(l)

    # Loads the relevant input matrix for the defined Kbas.
    X = spi.loadmat(work_dir+'X_'+str(Kbas)+'.mat')['X']

    # Loads the fidelity values.
    y = spi.loadmat(work_dir+'y_'+str(Kbas)+'.mat')['y']
    y = y[:,1].reshape([2*m,1])
    y = np.abs(y)

    if os.path.isdir(stor_dir)==False:
        os.mkdir(stor_dir)

    # --------------------------------------------------------------------------
    # Data preprocessing and splitting to 'train', 'validation' and 'test' sets.
    # --------------------------------------------------------------------------  

    # Splits the full dataset into training, validation and test subsets.
    x_train0, x_rest, y_train, y_rest = train_test_split(X, y, test_size=0.2, shuffle=True)
    x_test, x_val, y_test, y_val = train_test_split(x_rest, y_rest, test_size=0.5, shuffle=True)

    # Performs standard data normalization and scaling.
    x_train = preprocessing.StandardScaler().fit(x_train0).transform(x_train0)
    x_train = x_train.astype('float32')
    x_test = preprocessing.StandardScaler().fit(x_train0).transform(x_test)
    x_test = x_test.astype('float32')
    x_val = preprocessing.StandardScaler().fit(x_train0).transform(x_val)
    x_val = x_val.astype('float32')  

    # ---------------------------------------------
    # FidNet model definition and parametrization.
    # ---------------------------------------------

    # Defines gradient optimization batch size and training epochs.
    batch_size = 1024
    epochs = 500
    
    tf.keras.initializers.he_normal()

    # Converts inputs to single-channel 2D images.
    img_rows = int(np.ceil(np.power(x_train.shape[1],0.5)))
    img_cols = img_rows
    x_train = np.concatenate((x_train,np.zeros((x_train.shape[0],img_rows*img_cols-x_train.shape[1]))),axis=1)
    x_test = np.concatenate((x_test,np.zeros((x_test.shape[0],img_rows*img_cols-x_test.shape[1]))),axis=1)
    x_val = np.concatenate((x_val,np.zeros((x_val.shape[0],img_rows*img_cols-x_val.shape[1]))),axis=1)

    K.clear_session()
    x_train=x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test=x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    x_val=x_val.reshape(x_val.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1) 
    
    # Sets up neural network
    X_input = Input(input_shape) 

    Xnn = Conv2D(4, kernel_size = (5,5), padding='valid',strides=1, kernel_initializer = 'he_uniform')(X_input)
    Xnn = BatchNormalization()(Xnn)
    Xnn = Activation('relu')(Xnn)
    Xnn = Dropout(0.3)(Xnn)

    Xnn = Conv2D(8, kernel_size = (5,5), padding='valid',strides=1, kernel_initializer = 'he_uniform')(Xnn)
    Xnn = BatchNormalization()(Xnn)
    Xnn = Activation('relu')(Xnn)
    Xnn = Dropout(0.5)(Xnn)

    Xnn = Conv2D(16, kernel_size = (5,5), padding='valid',strides=1, kernel_initializer = 'he_uniform')(Xnn)
    Xnn = BatchNormalization()(Xnn)
    Xnn = Activation('relu')(Xnn)
    Xnn = Dropout(0.5)(Xnn)

    Xnn = AveragePooling2D(pool_size=(2,2),strides=2)(Xnn)

    Xnn = Flatten()(Xnn)

    Xnn = Dense(32)(Xnn)
    Xnn = BatchNormalization()(Xnn)
    Xnn = Activation('sigmoid')(Xnn)
    Xnn = Dropout(0.5)(Xnn)

    Xnn = Dense(num_classes,activation='sigmoid')(Xnn)

    model = Model(inputs = X_input, outputs = Xnn)

    model.summary()

    model.compile(loss='mse',optimizer='nadam')

    # -----------------
    # Network training.
    # -----------------

    # Sets up the scoreboard and checkpoint monitor to record training results.    
    checkpointer = ModelCheckpoint(filepath=stor_dir+"model_FidNet_K_"+str(Kbas)+".h5",
                                   verbose=0, monitor='val_loss', mode='min', save_best_only=True)
    tensorboard = TensorBoard(log_dir='./logs',
                              histogram_freq=0,
                              write_graph=True,
                              write_images=True)

    # Trains ICCNet with training and validation data subsets and records all results.
    history = model.fit(x_train, y_train,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(x_val, y_val),
                        verbose=1,
                        callbacks=[checkpointer, tensorboard]).history

    with open(stor_dir+"model_FidNet_K_"+str(Kbas)+"_hist.txt", 'w') as outfile:  
        json.dump(history, outfile)

    # Evaluates the trained model with test data subsets.
    score=model.evaluate(x_test,y_test,verbose=0)

    print('Test loss:',score)

    # Stores all split data subsets into a dictionary and saves it into a MATLAB cell file.
    Xy_dict = {'x_val': x_val, 'y_val': y_val, 'x_test': x_test, 'y_test': y_test, 'x_train0': x_train0, 'y_train': y_train}
    spi.savemat(stor_dir+"model_FidNet_K_"+str(Kbas)+"_Xy.mat", Xy_dict)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 23, 23, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 19, 19, 4)         104       
_________________________________________________________________
batch_normalization_1 (Batch (None, 19, 19, 4)         16        
_________________________________________________________________
activation_1 (Activation)    (None, 19, 19, 4)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 19, 19, 4)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 15, 15, 8)         808       
_________________________________________________________________
batch_normalization_2 (Batch (None, 15, 15, 8)         32        
__________

Epoch 128/500
6400/6400 [==============================] - 0s 21us/step - loss: 0.0080 - val_loss: 0.0042
Epoch 129/500
6400/6400 [==============================] - 0s 21us/step - loss: 0.0081 - val_loss: 0.0042
Epoch 130/500
6400/6400 [==============================] - 0s 20us/step - loss: 0.0079 - val_loss: 0.0041
Epoch 131/500
6400/6400 [==============================] - 0s 20us/step - loss: 0.0079 - val_loss: 0.0041
Epoch 132/500
6400/6400 [==============================] - 0s 19us/step - loss: 0.0078 - val_loss: 0.0041
Epoch 133/500
6400/6400 [==============================] - 0s 21us/step - loss: 0.0076 - val_loss: 0.0040
Epoch 134/500
6400/6400 [==============================] - 0s 21us/step - loss: 0.0078 - val_loss: 0.0040
Epoch 135/500
6400/6400 [==============================] - 0s 21us/step - loss: 0.0078 - val_loss: 0.0040
Epoch 136/500
6400/6400 [==============================] - 0s 21us/step - loss: 0.0076 - val_loss: 0.0039
Epoch 137/500
6400/6400 [=====================

6400/6400 [==============================] - 0s 22us/step - loss: 0.0050 - val_loss: 0.0034
Epoch 283/500
6400/6400 [==============================] - 0s 21us/step - loss: 0.0050 - val_loss: 0.0034
Epoch 284/500
6400/6400 [==============================] - 0s 20us/step - loss: 0.0052 - val_loss: 0.0034
Epoch 285/500
6400/6400 [==============================] - 0s 20us/step - loss: 0.0050 - val_loss: 0.0034
Epoch 286/500
6400/6400 [==============================] - 0s 22us/step - loss: 0.0050 - val_loss: 0.0034
Epoch 287/500
6400/6400 [==============================] - 0s 20us/step - loss: 0.0051 - val_loss: 0.0034
Epoch 288/500
6400/6400 [==============================] - 0s 22us/step - loss: 0.0052 - val_loss: 0.0034
Epoch 289/500
6400/6400 [==============================] - 0s 20us/step - loss: 0.0052 - val_loss: 0.0034
Epoch 290/500
6400/6400 [==============================] - 0s 21us/step - loss: 0.0051 - val_loss: 0.0034
Epoch 291/500
6400/6400 [==============================] - 0

6400/6400 [==============================] - 0s 22us/step - loss: 0.0044 - val_loss: 0.0033
Epoch 437/500
6400/6400 [==============================] - 0s 21us/step - loss: 0.0045 - val_loss: 0.0033
Epoch 438/500
6400/6400 [==============================] - 0s 20us/step - loss: 0.0044 - val_loss: 0.0033
Epoch 439/500
6400/6400 [==============================] - 0s 21us/step - loss: 0.0043 - val_loss: 0.0033
Epoch 440/500
6400/6400 [==============================] - 0s 21us/step - loss: 0.0045 - val_loss: 0.0033
Epoch 441/500
6400/6400 [==============================] - 0s 20us/step - loss: 0.0045 - val_loss: 0.0033
Epoch 442/500
6400/6400 [==============================] - 0s 19us/step - loss: 0.0044 - val_loss: 0.0033
Epoch 443/500
6400/6400 [==============================] - 0s 20us/step - loss: 0.0043 - val_loss: 0.0033
Epoch 444/500
6400/6400 [==============================] - 0s 20us/step - loss: 0.0044 - val_loss: 0.0034
Epoch 445/500
6400/6400 [==============================] - 0

Train on 6400 samples, validate on 800 samples
Epoch 1/500
6400/6400 [==============================] - 1s 204us/step - loss: 0.0317 - val_loss: 0.0108
Epoch 2/500
6400/6400 [==============================] - 0s 35us/step - loss: 0.0286 - val_loss: 0.0123
Epoch 3/500
6400/6400 [==============================] - 0s 34us/step - loss: 0.0253 - val_loss: 0.0154
Epoch 4/500
6400/6400 [==============================] - 0s 32us/step - loss: 0.0245 - val_loss: 0.0149
Epoch 5/500
6400/6400 [==============================] - 0s 34us/step - loss: 0.0237 - val_loss: 0.0192
Epoch 6/500
6400/6400 [==============================] - 0s 34us/step - loss: 0.0228 - val_loss: 0.0153
Epoch 7/500
6400/6400 [==============================] - 0s 34us/step - loss: 0.0228 - val_loss: 0.0195
Epoch 8/500
6400/6400 [==============================] - 0s 34us/step - loss: 0.0228 - val_loss: 0.0140
Epoch 9/500
6400/6400 [==============================] - 0s 33us/step - loss: 0.0216 - val_loss: 0.0138
Epoch 10/500
640

...

6400/6400 [==============================] - 1s 99us/step - loss: 0.0489 - val_loss: 0.0580
Epoch 186/500
6400/6400 [==============================] - 1s 102us/step - loss: 0.0471 - val_loss: 0.0578
Epoch 187/500
6400/6400 [==============================] - 1s 99us/step - loss: 0.0485 - val_loss: 0.0578
Epoch 188/500
6400/6400 [==============================] - 1s 100us/step - loss: 0.0486 - val_loss: 0.0578
Epoch 189/500
6400/6400 [==============================] - 1s 97us/step - loss: 0.0482 - val_loss: 0.0578
Epoch 190/500
6400/6400 [==============================] - 1s 99us/step - loss: 0.0479 - val_loss: 0.0580
Epoch 191/500
6400/6400 [==============================] - 1s 97us/step - loss: 0.0477 - val_loss: 0.0582
Epoch 192/500
6400/6400 [==============================] - 1s 98us/step - loss: 0.0480 - val_loss: 0.0584
Epoch 193/500
6400/6400 [==============================] - 1s 96us/step - loss: 0.0487 - val_loss: 0.0583
Epoch 194/500
6400/6400 [==============================] -

Epoch 262/500
6400/6400 [==============================] - 1s 101us/step - loss: 0.0465 - val_loss: 0.0598
Epoch 263/500
6400/6400 [==============================] - 1s 99us/step - loss: 0.0473 - val_loss: 0.0600
Epoch 264/500
6400/6400 [==============================] - 1s 101us/step - loss: 0.0462 - val_loss: 0.0602
Epoch 265/500
6400/6400 [==============================] - 1s 100us/step - loss: 0.0468 - val_loss: 0.0602
Epoch 266/500
6400/6400 [==============================] - 1s 104us/step - loss: 0.0469 - val_loss: 0.0603
Epoch 267/500
6400/6400 [==============================] - 1s 99us/step - loss: 0.0464 - val_loss: 0.0605
Epoch 268/500
6400/6400 [==============================] - 1s 99us/step - loss: 0.0468 - val_loss: 0.0607
Epoch 269/500
6400/6400 [==============================] - 1s 100us/step - loss: 0.0464 - val_loss: 0.0609
Epoch 270/500
6400/6400 [==============================] - 1s 98us/step - loss: 0.0472 - val_loss: 0.0609
Epoch 271/500
6400/6400 [================

6400/6400 [==============================] - 1s 96us/step - loss: 0.0453 - val_loss: 0.0625
Epoch 340/500
6400/6400 [==============================] - 1s 99us/step - loss: 0.0451 - val_loss: 0.0626
Epoch 341/500
6400/6400 [==============================] - 1s 98us/step - loss: 0.0453 - val_loss: 0.0626
Epoch 342/500
6400/6400 [==============================] - 1s 99us/step - loss: 0.0451 - val_loss: 0.0626
Epoch 343/500
6400/6400 [==============================] - 1s 98us/step - loss: 0.0456 - val_loss: 0.0625
Epoch 344/500
6400/6400 [==============================] - 1s 98us/step - loss: 0.0457 - val_loss: 0.0623
Epoch 345/500
6400/6400 [==============================] - 1s 98us/step - loss: 0.0451 - val_loss: 0.0623
Epoch 346/500
6400/6400 [==============================] - 1s 101us/step - loss: 0.0457 - val_loss: 0.0624
Epoch 347/500
6400/6400 [==============================] - 1s 98us/step - loss: 0.0457 - val_loss: 0.0625
Epoch 348/500
6400/6400 [==============================] - 

Epoch 416/500
6400/6400 [==============================] - 1s 100us/step - loss: 0.0449 - val_loss: 0.0641
Epoch 417/500
6400/6400 [==============================] - 1s 100us/step - loss: 0.0444 - val_loss: 0.0641
Epoch 418/500
6400/6400 [==============================] - 1s 100us/step - loss: 0.0446 - val_loss: 0.0640
Epoch 419/500
6400/6400 [==============================] - 1s 101us/step - loss: 0.0446 - val_loss: 0.0642
Epoch 420/500
6400/6400 [==============================] - 1s 98us/step - loss: 0.0446 - val_loss: 0.0642
Epoch 421/500
6400/6400 [==============================] - 1s 97us/step - loss: 0.0442 - val_loss: 0.0643
Epoch 422/500
6400/6400 [==============================] - 1s 98us/step - loss: 0.0441 - val_loss: 0.0644
Epoch 423/500
6400/6400 [==============================] - 1s 101us/step - loss: 0.0450 - val_loss: 0.0645
Epoch 424/500
6400/6400 [==============================] - 1s 98us/step - loss: 0.0446 - val_loss: 0.0644
Epoch 425/500
6400/6400 [================

Epoch 493/500
6400/6400 [==============================] - 1s 101us/step - loss: 0.0429 - val_loss: 0.0661
Epoch 494/500
6400/6400 [==============================] - 1s 102us/step - loss: 0.0440 - val_loss: 0.0661
Epoch 495/500
6400/6400 [==============================] - 1s 100us/step - loss: 0.0439 - val_loss: 0.0661
Epoch 496/500
6400/6400 [==============================] - 1s 101us/step - loss: 0.0440 - val_loss: 0.0661
Epoch 497/500
6400/6400 [==============================] - 1s 98us/step - loss: 0.0433 - val_loss: 0.0662
Epoch 498/500
6400/6400 [==============================] - 1s 100us/step - loss: 0.0441 - val_loss: 0.0662
Epoch 499/500
6400/6400 [==============================] - 1s 99us/step - loss: 0.0434 - val_loss: 0.0663
Epoch 500/500
6400/6400 [==============================] - 1s 101us/step - loss: 0.0439 - val_loss: 0.0664
Test loss: 0.07459545135498047
